<a href="https://colab.research.google.com/github/HarryPotter12/PractiseML/blob/master/AML_Ass1_Q4_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 4. Decision Trees: (13 marks) 
In this question, you will use the Wine dataset, a popular dataset to evaluate classification algorithms. The classification task is to determine, based on various parameters, whether a wine quality is over 7. The dataset has already been preprocessed to convert this into a binary classification problem (scores less than 7 belong to the “zero” class, and scores greater than or equal to 7 belong to the “one” class). Each line describes a wine, using 12 columns: the first 11 describe the wine’s characteristics (details), and the last column is a ground truth label for the quality of the wine (0/1). You must not use the last column as an input feature when you classify the data.


#### (a) (5 marks) Decision Tree Implementation: 
Implement your own version of the decision tree using binary univariate split, entropy and information gain. (If you are using Python, you can use the skeleton code provided. For this question, use of libraries like sklearn is not permitted; you can use numpy or pandas as required though.)

In [ ]:
import csv
import math
import random

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Enter You Name Here
myname = "Anuran-Banerjee-" # or "Doe-Jane-"

In [ ]:
# Implement your decision tree below
class DecisionTree():
    tree = {}

    def learn(self, training_set, attributes, target):
        # implement this function
        self.tree = build_tree(training_set, attributes, target)

In [ ]:
# Class Node which will be used while classify a test-instance using the tree which was built earlier
class Node():
    value = ""
    children = []

    def __init__(self, val, dictionary):
        self.value = val
        if (isinstance(dictionary, dict)):
            #self.children = dictionary.keys()
            self.children = list(dictionary)

In [ ]:
# Majority Function which tells which class has more entries in given data-set
def majorClass(attributes, data, target):

    freq = {}
    index = attributes.index(target)

    for tuple in data:
        #if (freq.has_key(tuple[index])):
        if tuple[index] in freq:
            freq[tuple[index]] += 1 
        else:
            freq[tuple[index]] = 1

    max = 0
    major = ""

    #for key in freq.keys():
    for key in list(freq):
        if freq[key]>max:
            max = freq[key]
            major = key

    return major

In [ ]:
# Calculates the entropy of the data given the target attribute
def entropy(attributes, data, targetAttr):

    freq = {}
    dataEntropy = 0.0

    i = 0
    for entry in attributes:
        if (targetAttr == entry):
            break
        i = i + 1

    i = i - 1

    for entry in data:
        #if (freq.has_key(entry[i])):
        if entry[i] in freq:
            freq[entry[i]] += 1.0
        else:
            freq[entry[i]]  = 1.0

    for freq in freq.values():
        dataEntropy += (-freq/len(data)) * math.log(freq/len(data), 2) 
        
    return dataEntropy

In [ ]:
# Calculates the information gain (reduction in entropy) in the data when a particular attribute is chosen for splitting the data.
def info_gain(attributes, data, attr, targetAttr):

    freq = {}
    subsetEntropy = 0.0
    i = attributes.index(attr)

    for entry in data:
        #if (freq.has_key(entry[i])):
        if entry[i] in freq:
            freq[entry[i]] += 1.0
        else:
            freq[entry[i]]  = 1.0

    #for val in freq.keys():
    for val in list(freq):
        valProb        = freq[val] / sum(freq.values())
        dataSubset     = [entry for entry in data if entry[i] == val]
        subsetEntropy += valProb * entropy(attributes, dataSubset, targetAttr)

    return (entropy(attributes, data, targetAttr) - subsetEntropy)

In [ ]:
# This function chooses the attribute among the remaining attributes which has the maximum information gain.
def attr_choose(data, attributes, target):

    best = attributes[0]
    maxGain = 0;

    for attr in attributes:
        newGain = info_gain(attributes, data, attr, target) 
        if newGain>maxGain:
            maxGain = newGain
            best = attr

    return best

In [ ]:
# This function will get unique values for that particular attribute from the given data
def get_values(data, attributes, attr):

    index = attributes.index(attr)
    values = []

    for entry in data:
        if entry[index] not in values:
            values.append(entry[index])

    return values

In [ ]:
# This function will get all the rows of the data where the chosen "best" attribute has a value "val"
def get_data(data, attributes, best, val):

    new_data = [[]]
    index = attributes.index(best)

    for entry in data:
        if (entry[index] == val):
            newEntry = []
            for i in range(0,len(entry)):
                if(i != index):
                    newEntry.append(entry[i])
            new_data.append(newEntry)

    new_data.remove([])    
    return new_data

In [ ]:
# This function is used to build the decision tree using the given data, attributes and the target attributes. It returns the decision tree in the end.
def build_tree(data, attributes, target):

    data = data[:]
    vals = [record[attributes.index(target)] for record in data]
    default = majorClass(attributes, data, target)

    if not data or (len(attributes) - 1) <= 0:
        return default
    elif vals.count(vals[0]) == len(vals):
        return vals[0]
    else:
        best = attr_choose(data, attributes, target)
        tree = {best:{}}
    
        for val in get_values(data, attributes, best):
            new_data = get_data(data, attributes, best, val)
            newAttr = attributes[:]
            newAttr.remove(best)
            subtree = build_tree(new_data, newAttr, target)
            tree[best][val] = subtree
    
    return tree

In [ ]:
def run_decision_tree():

    # Load data set
    with open("/content/gdrive/My Drive/Datasets/wine-dataset.csv") as f:
        attrib = next(f, None)
        data = [tuple(line) for line in csv.reader(f, delimiter=",")]
    print("Number of records: %d" % len(data))
    
    attributes = attrib.split(",")
    target = attributes[-1]
    # Split training/test sets
    # You need to modify the following code for cross validation.
    K = 10
    training_set = [x for i, x in enumerate(data) if i % K != 7]
    test_set = [x for i, x in enumerate(data) if i % K == 7]
    
    tree = DecisionTree()
    # Construct a tree using training set
    tree.learn( training_set, attributes , target )

    # Classify the test set using the tree we just constructed
    results = []
    # for instance in test_set:
    #     result = tree.classify( instance[:-1] )
    #     results.append( result == instance[-1])

    for instance in test_set:
        tempDict = tree.tree.copy()
        result = ""
        while(isinstance(tempDict, dict)):
            #root = Node(tempDict.keys()[0], tempDict[tempDict.keys()[0]])
            root = Node(list(tempDict)[0], tempDict[list(tempDict)[0]])
            #tempDict = tempDict[tempDict.keys()[0]]
            tempDict = tempDict[list(tempDict)[0]]
            index = attributes.index(root.value)
            value = instance[index]
            #if(value in tempDict.keys()):
            if(value in list(tempDict)):
                child = Node(value, tempDict[value])
                result = tempDict[value]
                tempDict = tempDict[value]
            else:
                result = "Null"
                break
        if result != "Null":
            results.append(result == instance[-1])
            
    # Accuracy
    accuracy = float(results.count(True))/float(len(results))
    print("=== Initial Implementation ===")
    print("Accuracy: %.4f" % accuracy)
    
    # Writing results to a file 
    f = open("/content/gdrive/My Drive/"+myname+"result.txt", "w")
    f.write("=== Initial Implementation === \n")
    f.write("Accuracy: %.4f\n" % accuracy)
    f.close()
    
    K = 10
    acc = []
    for k in range(K):
        # random.shuffle(data)
        training_set = [x for i, x in enumerate(data) if i % K != k]
        test_set = [x for i, x in enumerate(data) if i % K == k]
        tree = DecisionTree()
        tree.learn( training_set, attributes, target )
        results = []

        for entry in test_set:
            tempDict = tree.tree.copy()
            result = ""
            while(isinstance(tempDict, dict)):
                #root = Node(tempDict.keys()[0], tempDict[tempDict.keys()[0]])
                root = Node(list(tempDict)[0], tempDict[list(tempDict)[0]])
                #tempDict = tempDict[tempDict.keys()[0]]
                tempDict = tempDict[list(tempDict)[0]]
                index = attributes.index(root.value)
                value = entry[index]
                #if(value in tempDict.keys()):
                if(value in list(tempDict)):
                    child = Node(value, tempDict[value])
                    result = tempDict[value]
                    tempDict = tempDict[value]
                else:
                    result = "Null"
                    break
            if result != "Null":
                results.append(result == entry[-1])

        accuracy = float(results.count(True))/float(len(results))
        acc.append(accuracy)

    avg_acc = sum(acc)/len(acc)
    print("=== After Cross-validation ===")
    print("Average accuracy: %.4f" % avg_acc)    

    # Writing results to a file
    f = open("/content/gdrive/My Drive/"+myname+"result.txt", "a")
    f.write("=== After Cross-validation === \n")
    f.write("Average accuracy: %.4f\n" % avg_acc)
    f.close()


if __name__ == "__main__":
  run_decision_tree()

In [ ]:
run_decision_tree()

Number of records: 4898
=== Initial Implementation ===
Accuracy: 0.8910
=== After Cross-validation ===
Average accuracy: 0.9015
